# Handle the 'data' of the fishes
Wrestle with the data, check parameters and generate some helping files

In [ ]:
import platform
import os
import glob
import pandas
import imageio
import numpy
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import seaborn
import dask
import dask_image.imread
from dask.distributed import Client, LocalCluster
import skimage
from tqdm import notebook

In [ ]:
# Set dask temporary folder
# Do this before creating a client: https://stackoverflow.com/a/62804525/323100
import tempfile
if 'Linux' in platform.system():
    tmp = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
elif 'Darwin' in platform.system():
    tmp = tempfile.gettempdir()
else:
    if 'anaklin' in platform.node():
        tmp = os.path.join('F:\\')
    else:
        tmp = os.path.join('D:\\')
dask.config.set({'temporary_directory': os.path.join(tmp, 'tmp')})
print('Dask temporarry files go to %s' % dask.config.get('temporary_directory'))

In [ ]:
# Start cluster and client now, after setting tempdir
cluster = LocalCluster(n_workers=8)
client = Client(cluster)

In [ ]:
print('You can seee what DASK is doing at "http://localhost:%s/status"' % client.scheduler_info()['services']['dashboard'])

In [ ]:
# # Ignore warnings in the notebook
# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
# Set up figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
plt.rcParams['figure.figsize'] = (16, 9)  # Size up figures a bit
plt.rcParams['figure.dpi'] = 200

In [ ]:
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'

In [ ]:
# Display all plots identically
lines = 3
# And then do something like
# plt.subplot(lines, numpy.ceil(len(Data) / float(lines)), c + 1)

In [ ]:
# Different locations if running either on Linux or Windows
FastSSD = False
overthere = True # Load the data directly from the iee-research_storage drive
# to speed things up significantly
if 'Linux' in platform.system():
    if FastSSD:
        BasePath = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
    else:
        BasePath = os.path.join(os.sep, 'home', 'habi', '1272')
elif 'Darwin' in platform.system():
    FastSSD = False
    BasePath = os.path.join('/Users/habi/Dev/EAWAG/Data')
elif 'Windows' in platform.system():
    if FastSSD:
        BasePath = os.path.join('F:\\')
    else:
        if 'anaklin' in platform.node():
            BasePath = os.path.join('S:\\')
        else:
            BasePath = os.path.join('D:\\Results')
Root = os.path.join(BasePath, 'EAWAG')
if overthere:       
    if 'Linux' in platform.system():
        Root = os.path.join(os.sep, 'home', 'habi', 'research-storage-iee')
    else:
        Root = os.path.join('I:\\microCTupload')
print('We are loading all the data from %s' % Root)

In [ ]:
def get_pixelsize(logfile):
    """Get the pixel size from the scan log file"""
    pixelsize=None    
    with open(logfile, 'r') as f:
        for line in f:
            if 'Image Pixel' in line and 'Scaled' not in line:
                pixelsize = float(line.split('=')[1])
    return(pixelsize)

In [ ]:
def get_projectionsize(logfile):
    """How big did we set the camera?"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Number Of Rows' in line:
                y = int(line.split('=')[1])
            if 'Number Of Columns' in line:
                x = int(line.split('=')[1])                
    return(x*y)

In [ ]:
def get_filter(logfile):
    """Get the filter we used whole scanning from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Filter=' in line:
                whichfilter = line.split('=')[1].strip()
    return(whichfilter)

In [ ]:
def get_exposuretime(logfile):
    """Get the exposure time size from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Exposure' in line:
                exposuretime = int(line.split('=')[1])
    return(exposuretime)

In [ ]:
def get_ringartefact(logfile):
    """Get the ring artefact correction from the  scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Ring Artifact' in line:
                ringartefactcorrection = int(line.split('=')[1])
    return(ringartefactcorrection)

In [ ]:
def get_reconstruction_grayvalue(logfile):
    grayvalue = None
    """How did we map the brightness of the reconstructions?"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Maximum for' in line:
                grayvalue = float(line.split('=')[1])
    return(grayvalue)

In [ ]:
def get_beamhardening(logfile):
    """Get the beamhardening correction from the  scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Hardening' in line:
                beamhardeningcorrection = int(line.split('=')[1])
    return(beamhardeningcorrection)

In [ ]:
def get_rotationstep(logfile):
    """Get the rotation step from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Rotation Step' in line:
                rotstep = float(line.split('=')[1])
    return(rotstep)

In [ ]:
def get_frameaveraging(logfile):
    """Get the frame averaging from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Averaging' in line:
                avg = line.split('=')[1]
    return(avg)

In [ ]:
def get_machine(logfile):
    """Get the machine we used to scan"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scanner' in line:
                machine = line.split('=')[1].strip()
    return(machine)

In [ ]:
def get_scantime(logfile):
    """How long did we scan?"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scan duration' in line:
                time = line.split('=')[1].strip()
    return(pandas.to_timedelta(time))

In [ ]:
def get_stacks(logfile):
    """How many stacks/connected scans did we make?"""
    stacks = 1
    with open(logfile, 'r') as f:
        for line in f:
            if 'conn' in line:
                stacks = int(line.split('=')[1])
    return(stacks)

In [ ]:
def get_scandate(logfile, verbose=False):
    """When did we scan the fish?"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Study Date and Time' in line:
                if verbose:
                    print('Found "date" line: %s' % line.strip())
                datestring = line.split('=')[1].strip().replace('  ', ' ')
                if verbose:
                    print('The date string is: %s' % datestring)
                date = pandas.to_datetime(datestring , format='%d %b %Y %Hh:%Mm:%Ss')
                if verbose:
                    print('Parsed to: %s' % date)
                (date)
    return(date.isoformat())

In [ ]:
def get_git_hash():
    '''
    Get the current git hash from the repository.
    Based on http://stackoverflow.com/a/949391/323100 and
    http://stackoverflow.com/a/18283905/323100
    '''
    from subprocess import Popen, PIPE
    import os
    gitprocess = Popen(['git',
                        '--git-dir',
                        os.path.join(os.getcwd(), '.git'),
                        'rev-parse',
                        '--short',
                        '--verify',
                        'HEAD'],
                       stdout=PIPE)
    (output, _) = gitprocess.communicate()
    return output.strip().decode("utf-8")

In [ ]:
# # Make directory for output
# OutPutDir = os.path.join(os.getcwd(), 'Output', get_git_hash())
# print('We are saving all the output to %s' % OutPutDir)
# os.makedirs(OutPutDir, exist_ok=True)

In [ ]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [ ]:
# Get *all* log files
# Sort them by time, not name
Data['LogFile'] = [f for f in sorted(glob.glob(os.path.join(Root, '**', '*.log'),
                                               recursive=True),
                                     key=os.path.getmtime)]

In [ ]:
# Get all folders
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]

In [ ]:
# Check for samples which are not yet reconstructed
for c, row in Data.iterrows():
    # Iterate over every 'proj' folder
    if 'proj' in row.Folder:
        if not 'TScopy' in row.Folder and not 'PR' in row.Folder:
            # If there's nothing with 'rec*' on the same level, then tell us        
            if not glob.glob(row.Folder.replace('proj', 'rec')):
                print(glob.glob(row.Folder.replace('proj', 'rec')))
                print('- %s is missing matching reconstructions' % row.LogFile[len(Root)+1:])

In [ ]:
# Get rid of all non-rec logfiles
for c, row in Data.iterrows():
    if 'rec' not in row.Folder:
        Data.drop([c], inplace=True)
    elif 'rectmp.log' in row.LogFile:
        Data.drop([c], inplace=True)
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [ ]:
Data

In [ ]:
# Generate us some meaningful colums
Data['Fish'] = [l[len(Root)+1:].split(os.sep)[0] for l in Data['LogFile']]
Data['Scan'] = ['_'.join(l[len(Root)+1:].split(os.sep)[1:-1]) for l in Data['LogFile']]

In [ ]:
# Get the file names of the reconstructions
Data['Reconstructions'] = [sorted(glob.glob(os.path.join(f, '*rec0*.png'))) for f in Data['Folder']]
Data['Number of reconstructions'] = [len(r) for r in Data.Reconstructions]

In [ ]:
# Get the file names of the reconstructions
for c, row in Data[Data['Number of reconstructions'] == 0].iterrows():
    print('%s/%s: Scan %s does not contain any reconstructions and '
          'will be removed in the next step' % (c+1, len(Data), os.path.join(row.Fish, row.Scan)))

In [ ]:
# Drop samples which have not been reconstructed yet
# Based on https://stackoverflow.com/a/13851602
# for c,row in Data.iterrows():
#     if not row['Number of reconstructions']:
#         print('%s contains no PNG files, we might be currently reconstructing it' % row.Folder)
print('We have %s folders in total' % (len(Data)))
Data = Data[Data['Number of reconstructions'] > 0]
Data.reset_index(drop=True, inplace=True)
print('Of which %s folders do contain reconstructions' % (len(Data)))

In [ ]:
# Get parameters to doublecheck from logfiles
Data['Voxelsize'] = [get_pixelsize(log) for log in Data['LogFile']]
Data['Filter'] = [get_filter(log) for log in Data['LogFile']]
Data['Exposuretime'] = [get_exposuretime(log) for log in Data['LogFile']]
Data['Scanner'] = [get_machine(log) for log in Data['LogFile']]
Data['Averaging'] = [get_frameaveraging(log) for log in Data['LogFile']]
Data['ProjectionSize'] = [get_projectionsize(log) for log in Data['LogFile']]
Data['RotationStep'] = [get_rotationstep(log) for log in Data['LogFile']]
Data['CameraWindow'] = [round((ps ** 0.5)/100)*100  for ps in Data['ProjectionSize']]
Data['Grayvalue'] = [get_reconstruction_grayvalue(log) for log in Data['LogFile']]
Data['RingartefactCorrection'] = [get_ringartefact(log) for log in Data['LogFile']]
Data['BeamHardeningCorrection'] = [get_beamhardening(log) for log in Data['LogFile']]
Data['Scan date'] = [get_scandate(log) for log in Data['LogFile']]
Data['Scan time'] = [get_scantime(log) for log in Data['LogFile']]
Data['Stacks'] = [get_stacks(log) for log in Data['LogFile']]

In [ ]:
Data['Scan time total'] = [ st * stk  for st, stk in zip(Data['Scan time'], Data['Stacks'])]

In [ ]:
# Generate a text file for each rec-folder, in which we can note what's going on with the fish
# Generate filename
for c,row in Data.iterrows():
    Data.at[c, 'CommentFile'] = os.path.join(os.path.dirname(row.Folder),
                                              row.Fish + '.' + row.Scan + '.md')
# Create actual file on disk
for c,row in Data.iterrows():
    # Only do this if the file does not already exist
    if not os.path.exists(row.CommentFile):
        with open(row.CommentFile, 'w', encoding='utf-8') as f:
            f.write('# Fish %s, Scan %s\n\n' % (row.Fish, row.Scan))
            f.write('This fish was scanned on %s on the %s, with a voxel size of %s μm.\n\n'
                    % (row['Scan date'], row.Scanner, numpy.round(row.Voxelsize, 2)))
            f.write('## Comments')

In [ ]:
# # https://www.geeksforgeeks.org/iterating-over-rows-and-columns-in-pandas-dataframe/
# columns = list(Data)
# columns.remove('Folder') 
# columns.remove('Fish')
# columns.remove('LogFile')
# columns.remove('Reconstructions')
# columns.remove('Number of reconstructions')
# columns.remove('Grayvalue')
# columns.remove('Scan time')
# columns.remove('Scan time total')
# columns.remove('Scan date')
# print(columns)
# for col in columns:
#     print(col)
#     print(Data[col].unique())
#     print(80*'-')    

In [ ]:
# # Check voxel sizes (*rounded* to two after-comma values)
# # If different, spit out which values
# roundto = 2
# if len(Data['Voxelsize'].round(roundto).unique()) > 1:
#     print('We scanned all datasets with %s different voxel sizes' % len(Data['Voxelsize'].round(roundto).unique()))
#     for vs in sorted(Data['Voxelsize'].round(roundto).unique()):
#         print('-', vs, 'um for ', end='')
#         for c, row in Data.iterrows():
#             if float(vs) == round(row['Voxelsize'], roundto):
#                 print(os.path.join(row['Fish'], row['Scan']), end=', ')
#         print('')
# else:
#     print('We scanned all datasets with equal voxel size, namely %s um.' % float(Data['Voxelsize'].round(roundto).unique()))

In [ ]:
# if len(Data['Grayvalue'].unique()) > 1:
#     print('We reconstructed the datasets with different maximum gray values, namely')
#     for gv in Data['Grayvalue'].unique():
#         print(gv, 'for Samples ', end='')
#         for c, row in Data.iterrows():
#             if float(gv) == row['Grayvalue']:
#                 print(os.path.join(row['Fish'], row['Scan']), end=', ')
#         print('')
# else:
#     print('We reconstructed all datasets with equal maximum gray value, namely %s.' % Data['Grayvalue'].unique()[0])

In [ ]:
# Data[['Fish', 'Scan',
#       'Voxelsize', 'Scanner',
#       'Scan date', 'CameraWindow', 'RotationStep', 'Averaging',
#       'Scan time', 'Stacks', 'Scan time total']]

In [ ]:
# Get an overview over the total scan time
# Nice output based on https://stackoverflow.com/a/8907407/323100
total_seconds = int(Data['Scan time total'].sum().total_seconds())
hours, remainder = divmod(total_seconds,60*60)
minutes, seconds = divmod(remainder,60)
print('In total, we scanned for %s hours and %s minutes)' % (hours, minutes))
for machine in Data['Scanner'].unique():
    total_seconds = int(Data[Data['Scanner'] == machine]['Scan time total'].sum().total_seconds())
    hours, remainder = divmod(total_seconds,60*60)
    minutes, seconds = divmod(remainder,60)
    print('\t - Of these, we scanned %s hours and %s minutes on the %s,'
          'for %s scans' % (hours,
                            minutes,
                            machine,
                            len(Data[Data['Scanner'] == machine])))

In [ ]:
Data.Fish.unique()

In [ ]:
len(Data)

In [ ]:
len(Data[Data['Scan'].str.contains('head')])

In [ ]:
Data[['Fish', 'Scan',
      'Voxelsize', 'Scanner',
      'Scan date', 'CameraWindow', 'RotationStep', 'Averaging', 'Scan time', 'Stacks' ]].to_excel('Details.xlsx')

In [ ]:
Data[['Fish', 'Scan',
      'Voxelsize', 'Scanner',
      'Scan date', 'CameraWindow',
      'RotationStep', 'Averaging', 'Scan time', 'Stacks' ]].to_excel(os.path.join(Root,'Details.xlsx'))

In [ ]:
# Read Mikkis datafile
MikkisFile = sorted(glob.glob(os.path.join(Root, 'X_ArchiveFiles', '*CTscanFishList.xlsx')))[0]
# Read excel file and use the first column as index
print('Reading in %s' % MikkisFile)
DataMikki = pandas.read_excel(MikkisFile)

In [ ]:
DataMikki.head()

In [ ]:
# Find the fish we look at and display all the info we know about it
# Set a substring you're looking for to the variable below
# In which jar can we find it?
fish = '104061'

In [ ]:
# In which jar should it be/go?
foundfishes = 0
for d, row in DataMikki.iterrows():
    if (str(fish).lower() in str(row.Fishec).lower()) or \
    (str(fish).lower() in str(row.FieldID).lower()) or \
    (str(fish).lower() in str(row.OtherID).lower()) or \
    (str(fish).lower() in str(row.ReplacementID).lower()):
        foundfishes = (row.Fishec, row.FieldID, row.OtherID, row.ReplacementID)
        # remove nan from the list of hits
        foundfishes = [str(x).lower() for x in foundfishes if pandas.isnull(x) == False]
        print('*%s*: The fish ' % fish, end='')        
        if len(foundfishes) > 1:
            for found in foundfishes:
                print(found.upper(), end='/')
        else:
            print(foundfishes[0].upper(), end='')
        print(' should now go in jar "length=%s cm" (%s))' % (row['Length(cm)'],
                                                              row['TemporaryJar']))
if not foundfishes:
    print('*%s*: Nothing found in %s' % (fish, MikkisFile))

In [ ]:
# Do we have something from this fish on disk?
ondisk = glob.glob(os.path.join(Root, '*%s*' % fish))
if len(ondisk):
    for found in ondisk:
        print('*%s*: Found on disk in %s' % (fish, found))
        foundondisk = 1
else:
    print('*%s*: Nothing found in %s' % (fish, Root))
    foundondisk = 0

In [ ]:
# Did we scan it already?
found = 0
for c, row in Data.iterrows():
    if fish in row.Fish:
        print('*%s*: Sample %s/%s was scanned on %s' % (fish, row['Fish'], row['Scan'], row['Scan date']))
        found = 1
if not found:
    if foundondisk:
        print('*%s*: We have a folder (%s) for this sample, but nothing in the dataframe, so it probably is all good' % (fish, ondisk[0]))
        print('Check the folder to be shure')
    else:
        print('*%s*: Nothing about this sample is found in our dataframe' % fish)

In [ ]:
# Can we find it in FullHeadList.txt?
def findinFullHeadList(sample):
    ''' Look for the sample in the FullHeadList.txt file'''
    fullheadlist = glob.glob(os.path.join(Root, 'FullHeadList.*'))[0]    
    found = 0
    with open(fullheadlist, 'r') as f:
        for line in f:
            if str(sample) in line:
                print(line.strip())
                found = 1
    if not found:
        return('*%s*: Nothing found in %s' % (sample, fullheadlist))
    else:
        return(None)
findinFullHeadList(fish)

In [ ]:
# Do we need to rescan this fish
# Find all relevant comment files
commentfiles = glob.glob(os.path.join(Root, '*%s*' % fish, '**', '*.md'), recursive=True)
print('We found these comment files in our dataframe')
for c, row in Data.iterrows():
    if fish in row.Fish:
        print('\t-', row.CommentFile)
        found = 1
print(80*'-')
if len(commentfiles):
    for commentfile in commentfiles:
        print('-', commentfile)
        print(10*'-')
        with open(commentfile, 'r', encoding='utf-8') as file:
            for line in file:
                print(line.strip())
                if 'rescan' in line:
                    print('BEEEEP!')
        print(80*'-')

60 of the fishes need complete head scans.
Let's try to go through Mikkis/Kassandras list and see how far we progressed through that list.

In [ ]:
# Read in full head list, go through all the scans we alredy did and see what needs to be done
fullheadlist = glob.glob(os.path.join(Root, '*Head*.txt'))[0]
HeadsToBeScanned = []
with open(fullheadlist, 'r', encoding='utf-8') as file:
    headdone = False
    for ln, line in enumerate(file):
        if line.strip():  #skip empty lines
            # The first 'item' on the line should be the fish ID
            fish = line.strip().split()[0].replace(',','').upper()
            # Let's ignore some lines which don't start with a fish ID
            # The set-join here removes duplicate characters from the string (e.g. =====, !! and ::)
            if len(''.join(set(fish))) > 2:
                for c, row in Data[Data.Fish == fish].iterrows():
                    if 'head' in row.Scan:
                        # print('\t%s has a head-scan' % row.Fish)
                        # print('%s has a head-scan on disk, and is found on line %s of the full head list' % (fish, ln + 1 ))
                        headdone = True
                    else:
                        headdone = False
                # At this point we have either found the fish in the list or 'headdone' is false
                if not headdone:
                    print('%s is missing a head-scan on disk, but is found on line %s of the full head list' % (fish, ln + 1 ))
                    HeadsToBeScanned.append(fish)             

In [ ]:
# Fish 10448 can be ignored because we did another scan after the head-scan, so we reset "headdone" in the loop above
# We could probably do it in a more clever way, but already spent too much time on this part :)
try:
    HeadsToBeScanned.remove('10448')
    # HeadsToBeScanned.remove('105515')
except ValueError:
    # Nothing to see here, pass along
    pass

In [ ]:
for fish in HeadsToBeScanned:
    # In which jar should we look for the fishes we still need to scan the head of?
    foundfishes = 0
    for d, row in DataMikki.iterrows():
        if (str(fish).lower() in str(row.Fishec).lower()) or \
        (str(fish).lower() in str(row.FieldID).lower()) or \
        (str(fish).lower() in str(row.OtherID).lower()) or \
        (str(fish).lower() in str(row.ReplacementID).lower()):
            foundfishes = (row.Fishec, row.FieldID, row.OtherID, row.ReplacementID)
            # remove nan from the list of hits
            foundfishes = [str(x).lower() for x in foundfishes if pandas.isnull(x) == False]
            print('*%s*: A fish called ' % fish, end='')        
            if len(foundfishes) > 1:
                for found in foundfishes:
                    print(found.upper(), end='/')
            else:
                print(foundfishes[0].upper(), end='')
            print(' should be found in jar "length=%s cm" (%s))' % (row['Length(cm)'],
                                                                    row['TemporaryJar']))
    if not foundfishes:
        print('*%s*: Nothing found in %s' % (fish, MikkisFile))

In [ ]:
# Some of the reconstructions need to be looked at?
# Mikki wrote something about this into the files.
# Get a list of *all* comment files
CommentFiles = glob.glob(os.path.join(Root, '**', '*.md'), recursive=True)

In [ ]:
# Read what we want
print('Going through all the %s comments files we find' % len(CommentFiles))
for c, cf in enumerate(CommentFiles):
    with open(cf, 'r', encoding='utf-8') as file:
        for line in file:
            if 'Mikki' in line:
                print('%03s/%s: %s: %s' % (c, len(CommentFiles), cf[len(Root)+1:], line.strip()))
            elif 'ML' in line:
                print('%03s/%s: %s: %s' % (c, len(CommentFiles), cf[len(Root)+1:], line.strip()))
            elif 'realign' in line:
                print('%03s/%s: %s: %s' % (c, len(CommentFiles), cf[len(Root)+1:], line.strip()))

In [ ]:
Data.sort_values(['Scan date'], ascending=False, inplace=True)